In [1]:
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#!pip install tensorflow_hub
import tensorflow_hub as hub
import numpy as np
import shutil
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import tensorboard
import os
import pandas as pd
import math

In [2]:
batch_size = 32
img_height = 224
img_width = 224

In [5]:
# In transfer learning, we do not need the last kayer of the model as it was trained for 1000 outputs.
# for this, set the parameter include_top = False. This means that we will use only the model architecture
# and do not need the last output layer from this architecture

# We also do not want the weights from imagenet training as we will perform the training for our own dataset.

In [4]:
dataset = 'Data/Processed_Data/train'
dataset_path = os.listdir(dataset)

print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

['train_ak_aug', 'train_bcc_aug', 'train_bkl_aug', 'train_df_aug', 'train_mel_aug', 'train_nv_aug', 'train_scc_aug', 'train_vasc_aug']
Types of classes labels found:  8


In [5]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# create a directory to save the files
create_dir("models")

In [3]:
# load the augmented data from the local data directory:

train_dir = "Data/Processed_Data/train"
test_dir = "Data/Processed_Data/test"
valid_dir = "Data/Processed_Data/valid_processed"

datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.2, data_format="channels_last", dtype=tf.float32) # The imported data will br normalized here.
test_datagen = ImageDataGenerator(rescale = 1./255, data_format="channels_last", dtype=tf.float32)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    shuffle = True,
    seed=123,subset = 'training') # set as training data

validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    shuffle = True,
    seed=123,subset = 'validation') # set as validation data

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    shuffle = True,
    seed=123,) # set as testing data

Found 57680 images belonging to 8 classes.
Found 14416 images belonging to 8 classes.
Found 3799 images belonging to 8 classes.


In [7]:
# Nasnet model
model = Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/nasnet_mobile/feature_vector/5",
               trainable=True, arguments=dict(batch_norm_momentum=0.997)),
    layers.Dense(8, activation='softmax'),
])

model.build([None, img_height, img_width, 3])  # Batch input shape.

In [8]:
METRICS = [
    keras.metrics.Accuracy(name= "accuracy"),
    keras.metrics.Precision(name = "precision"),
    keras.metrics.Recall(name = 'recall'),
    keras.metrics.AUC(name = 'auc'),
]
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS,)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1056)              4269716   
                                                                 
 dense (Dense)               (None, 8)                 8456      
                                                                 
Total params: 4,278,172
Trainable params: 4,241,434
Non-trainable params: 36,738
_________________________________________________________________


In [8]:
r = model.fit(
    train_generator,
    epochs = 30,
    steps_per_epoch=train_generator.samples//batch_size,
    validation_data = validation_generator,
    validation_steps=validation_generator.samples//batch_size,
)

Epoch 1/30
1802/1802 [==============================] - 1421s 747ms/step - loss: 0.9242 - accuracy: 5.4208e-05 - precision: 0.8333 - recall: 0.7512 - auc: 0.9779 - val_loss: 2.1613 - val_accuracy: 0.0014 - val_precision: 0.6208 - val_recall: 0.5810 - val_auc: 0.8849
Epoch 2/30
1802/1802 [==============================] - 1345s 747ms/step - loss: 0.6796 - accuracy: 3.9030e-04 - precision: 0.8943 - recall: 0.8499 - auc: 0.9904 - val_loss: 2.3130 - val_accuracy: 0.0019 - val_precision: 0.5803 - val_recall: 0.5340 - val_auc: 0.8555
Epoch 3/30
1802/1802 [==============================] - 1361s 755ms/step - loss: 0.5807 - accuracy: 7.4157e-04 - precision: 0.9108 - recall: 0.8758 - auc: 0.9930 - val_loss: 2.0663 - val_accuracy: 0.0023 - val_precision: 0.6726 - val_recall: 0.6410 - val_auc: 0.8888
Epoch 4/30
1802/1802 [==============================] - 1337s 742ms/step - loss: 0.5182 - accuracy: 0.0011 - precision: 0.9218 - recall: 0.8932 - auc: 0.9945 - val_loss: 1.8477 - val_accuracy: 8.3333

In [9]:
#model.save("models/model_Nasnet_30.h5")

In [4]:
#model = keras.models.load_model("models/model_Nasnet_30.h5",custom_objects={'KerasLayer':hub.KerasLayer})

In [5]:
#Confution Matrix and Classification Report
Y_pred = model.predict_generator(test_generator, test_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['ak', 'bcc', 'bkl', 'df', 'mel','nv', 'scc', 'vasc']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

C:\Users\GHAYAAS\AppData\Local\Temp/ipykernel_17544/2098712288.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_generator, test_generator.samples // batch_size+1)


Confusion Matrix
[[ 40   0   0  36   8  21  23   2]
 [118   5   0 145  29 100  95   6]
 [ 84   4   1 118  24  77  84   2]
 [  9   0   1  12   6   5   3   0]
 [160   4   0 198  37 131 145   3]
 [433  12   5 550 126 414 378  13]
 [ 24   1   0  28   4  18  19   0]
 [  8   0   0  11   3  11   5   0]]
Classification Report
              precision    recall  f1-score   support

          ak       0.05      0.31      0.08       130
         bcc       0.19      0.01      0.02       498
         bkl       0.14      0.00      0.00       394
          df       0.01      0.33      0.02        36
         mel       0.16      0.05      0.08       678
          nv       0.53      0.21      0.31      1931
         scc       0.03      0.20      0.04        94
        vasc       0.00      0.00      0.00        38

    accuracy                           0.14      3799
   macro avg       0.14      0.14      0.07      3799
weighted avg       0.34      0.14      0.18      3799



In [6]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_generator.classes, y_pred))

0.13898394314293236


In [7]:
from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(test_generator.classes, y_pred))

0.14058756116877513
